# Bài tập thực hành
```
ProtonX - NLP Class
```

### Hướng dẫn làm bài
- Trong bài tập này bạn sẽ sử dụng Python 3.
- Sau khi bạn viết Code của mình xong, hãy chạy dòng Code đó để xem kết quả bên dưới.

### [Quan trọng] Chú ý
- **Không sử dụng hàm `input()` tại bất kỳ dòng lệnh nào**
- **Không thay đổi dòng code return của hàm**

Các bạn sẽ thực hiện `code` trong các phần hiển thị `#TODO: Lập trình tại đây` và thay thế các vị trí `None`. Có những câu hỏi chỉ cần trả về đáp án.

Sau khi viết xong Code của bạn, bạn hãy ấn "SHIFT"+"ENTER" để thực hiện chạy lệnh của Cell đó.

---
Điểm số:
* 10 điểm / Câu

Tiêu chí chấm điểm:
* Các bài tập sẽ được chấm dựa trên các Test-case.
* Các bạn không khởi tạo lại giá trị đầu vào bên trong hàm. Có thể khởi tạo các giá trị này ngoài hàm nhằm mục đích kiểm thử.

**Mục tiêu của bài tập này:** Bài tập này hướng dẫn bạn cách sử dụng Embedding đã được đào tạo từ trước nhằm mục tiêu cải thiện kết quả cho model hiện tại.

Một số thông tin:
- Dữ liệu: IMDB Reviews với đầu vào là câu và nhãn là tích cực hay tiêu cực
- Mô hình phân loại nhị phân

## Tải Embedding


In [1]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip

--2024-03-10 12:52:55--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.51, 3.162.163.11, 3.162.163.34, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G   160MB/s    in 13s     

2024-03-10 12:53:09 (113 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]



In [2]:
!unzip /content/crawl-300d-2M.vec.zip -d /content

Archive:  /content/crawl-300d-2M.vec.zip
  inflating: /content/crawl-300d-2M.vec  


## Đọc embedding

In [3]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

def load_fasttext_embeddings(embedding_file):
    embeddings_index = {}
    with open(embedding_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

embedding_index = load_fasttext_embeddings('/content/crawl-300d-2M.vec')

In [4]:
embedding_index.get('player')



array([-3.056e-01, -1.920e-02,  8.140e-02, -2.653e-01,  1.441e-01,
       -1.800e-03, -1.189e-01,  6.810e-02,  1.431e-01,  3.468e-01,
        1.116e-01,  4.713e-01, -6.500e-02, -1.244e-01, -3.259e-01,
       -2.920e-02, -2.950e-02,  2.662e-01,  1.666e-01, -1.523e-01,
        2.065e-01,  2.094e-01, -3.650e-02,  6.600e-03,  1.083e-01,
        2.114e-01, -9.800e-03, -2.340e-02,  1.134e-01, -1.212e-01,
       -2.109e-01, -2.405e-01, -2.208e-01,  1.440e-02, -6.130e-02,
        4.092e-01, -2.209e-01,  2.242e-01, -9.540e-02,  1.917e-01,
        1.179e-01, -1.656e-01, -5.140e-02,  2.684e-01, -3.350e-02,
       -2.662e-01,  2.294e-01,  5.780e-02,  4.322e-01, -4.820e-02,
        3.831e-01, -8.190e-02,  3.509e-01, -1.817e-01, -1.810e-02,
       -2.055e-01,  2.816e-01, -2.247e-01, -4.130e-02, -1.292e-01,
        3.079e-01, -1.182e-01,  9.520e-02, -4.566e-01,  3.626e-01,
       -9.680e-02, -4.260e-02, -1.603e-01, -4.980e-02, -8.680e-02,
        2.094e-01, -2.611e-01,  2.800e-03,  4.930e-02, -1.820e

## Tải bộ dữ liệu

In [5]:
!wget https://storage.googleapis.com/protonx-cloud-storage/datasets/IMDB%20Dataset.csv
df = pd.read_csv('/content/IMDB Dataset.csv')

--2024-03-10 12:57:01--  https://storage.googleapis.com/protonx-cloud-storage/datasets/IMDB%20Dataset.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.207, 173.194.197.207, 173.194.198.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66212309 (63M) [text/csv]
Saving to: ‘IMDB Dataset.csv’

IMDB Dataset.csv    100%[===================>]  63.14M   119MB/s    in 0.5s    

2024-03-10 12:57:01 (119 MB/s) - ‘IMDB Dataset.csv’ saved [66212309/66212309]



In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**TODO 1:** Lập trình hàm lọc stopwords kết hợp với Lemmatization từ một câu bất kỳ
- Đầu vào: Một câu
- Đầu ra: Câu đã được tiền xử lý dữ liệu

In [7]:
import tensorflow as tf

In [8]:
import re

def process_string(text):
    # Tách các dấu câu và cách các từ xung quanh
    processed_text = re.sub(r'(?<=\S)([.,?!;:]+)(?=\s|\Z)', r' \1', text)
    return processed_text

# Ví dụ
text = "Tôi đi học, tôi đi học! Tôi đi học. Tôi đi học;"
processed_text = process_string(text)
print(processed_text)


Tôi đi học , tôi đi học ! Tôi đi học . Tôi đi học ;


In [9]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # text = tf.strings.lower(text)
    # text = tf.strings.regex_replace(text, '[.?!,]', r' \0 ')
    # text = tensor.numpy().decode('utf-8')
    text = re.sub(r'(?<=\S)([.,?!;:]+)(?=\s|\Z)', r' \1', text)
    text = " ".join(lemmatizer.lemmatize(token) for token in text.split() if token not in stop_words)
    # text = tf.strings.strip(text)
    return text

# Apply preprocessing to each review
df['processed_review'] = df['review'].apply(preprocess_text)


In [10]:
df['processed_review'].head()

0    One reviewer mentioned watching 1 Oz episode h...
1    A wonderful little production . <br /><br />Th...
2    I thought wonderful way spend time hot summer ...
3    Basically there's family little boy (Jake) thi...
4    Petter Mattei's "Love Time Money" visually stu...
Name: processed_review, dtype: object

In [11]:
# Test function
def test_preprocess_text():
    test_text = "The quick brown foxes were jumping over the lazy dogs."
    expected_output = "The quick brown fox jumping lazy dog ."  # Expected output may vary based on your lemmatization and stop words list
    processed_text = preprocess_text(test_text)

    assert processed_text == expected_output, f"Expected '{expected_output}', but got '{processed_text}'"

# Run the test
test_preprocess_text()
print("Test passed!")

Test passed!


Kết quả mong đợi sau xử lý

In [12]:
df.head()

,review,sentiment,processed_review
0,One of the other reviewers has mentioned that ...,positive,One reviewer mentioned watching 1 Oz episode h...
1,A wonderful little production. <br /><br />The...,positive,A wonderful little production . <br /><br />Th...
2,I thought this was a wonderful way to spend ti...,positive,I thought wonderful way spend time hot summer ...
3,Basically there's a family where a little boy ...,negative,Basically there's family little boy (Jake) thi...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"Petter Mattei's ""Love Time Money"" visually stu..."


In [13]:
def change_label(x):
  if x == 'positive':return 1
  else: return 0

# Chuyển nhãn thành giá trị số và thêm vào mảng
df['sentiment'] = df['sentiment'].apply(change_label)

In [14]:
raw_review = df['processed_review'].tolist()

In [15]:
raw_review[0]

"One reviewer mentioned watching 1 Oz episode hooked . They right , exactly happened me.<br /><br />The first thing struck Oz brutality unflinching scene violence , set right word GO . Trust , show faint hearted timid . This show pull punch regard drug , sex violence . Its hardcore , classic use word.<br /><br />It called OZ nickname given Oswald Maximum Security State Penitentary . It focus mainly Emerald City , experimental section prison cell glass front face inwards , privacy high agenda . Em City home many..Aryans , Muslims , gangsta , Latinos , Christians , Italians , Irish more....so scuffle , death stare , dodgy dealing shady agreement never far away.<br /><br />I would say main appeal show due fact go show dare . Forget pretty picture painted mainstream audience , forget charm , forget romance...OZ mess around . The first episode I ever saw struck nasty surreal , I say I ready , I watched , I developed taste Oz , got accustomed high level graphic violence . Not violence , inju

**TODO 2:**
- Thực hiện build bộ tách token từ cột processed_review
- Đồng thời chuyển nhãn cột sentiment về những giá trị số tương ứng
    - Tích cực: Giá trị 1
    - Tiêu cực: Giá trị 0

In [16]:
from keras.preprocessing.text import Tokenizer

In [17]:
labels = df['sentiment'].tolist()

In [18]:
# Xây dựng bộ Token
tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_review)
# Chuyển các câu hiện tại thành chuỗi số
sequence_review = tokenizer.texts_to_sequences(raw_review)

**TODO 3:** Padding cho chuỗi câu
- Chiều dài tối đa: 200
- Padding vào cuối câu
- Cắt ở cuối câu nếu câu dài hơn 200

In [19]:
data = pad_sequences(sequence_review, maxlen=200, truncating='post', padding='post')

In [20]:
data.shape

(50000, 200)

**TODO 4:** Chia bộ dữ liệu thành 2 phần theo tỉ lệ
- 80% cho training
- 20% cho validation

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
# Chia bộ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

**TODO 5:** Lập trình việc trích xuất embedding CBOW về một ma trận vector embedding của các từ trong bộ từ điển hiện tại

In [23]:
word_index = tokenizer.word_index

In [24]:
word_index

{'br': 1,
 'i': 2,
 'movie': 3,
 'the': 4,
 'film': 5,
 'one': 6,
 'like': 7,
 'this': 8,
 'time': 9,
 'good': 10,
 'it': 11,
 'character': 12,
 'story': 13,
 'even': 14,
 'get': 15,
 'would': 16,
 'make': 17,
 'see': 18,
 'really': 19,
 'well': 20,
 'and': 21,
 'scene': 22,
 'much': 23,
 'bad': 24,
 'great': 25,
 'also': 26,
 'people': 27,
 'first': 28,
 'way': 29,
 'show': 30,
 'but': 31,
 'made': 32,
 'thing': 33,
 'a': 34,
 'think': 35,
 'could': 36,
 'life': 37,
 'in': 38,
 'go': 39,
 'know': 40,
 'watch': 41,
 'love': 42,
 'two': 43,
 'plot': 44,
 'many': 45,
 'seen': 46,
 'never': 47,
 'say': 48,
 'end': 49,
 'acting': 50,
 'actor': 51,
 'look': 52,
 'best': 53,
 "it's": 54,
 'year': 55,
 'little': 56,
 'ever': 57,
 'if': 58,
 'better': 59,
 'take': 60,
 'man': 61,
 'come': 62,
 'still': 63,
 'work': 64,
 'there': 65,
 'he': 66,
 'part': 67,
 'something': 68,
 'find': 69,
 'back': 70,
 'want': 71,
 'give': 72,
 'lot': 73,
 'real': 74,
 "i'm": 75,
 'director': 76,
 'watching': 77

In [25]:
import numpy as np
def prepare_embedding_matrix(tokenizer_word_index, embedding_index, embedding_dim=300):
    """
    Prepares the embedding matrix.

    Parameters:
    - tokenizer_word_index: dict, a dictionary containing word-index pairs from tokenizer
    - embedding_index: dict, a dictionary containing word-embedding vector pairs
    - embedding_dim: int, the dimension of the embedding vectors

    Returns:
    - embedding_matrix: numpy.ndarray, the prepared embedding matrix
    """
    number_word = len(tokenizer_word_index) + 1
    embedding_matrix = np.zeros((number_word, embedding_dim))
    for word, idx in tokenizer_word_index.items():
      word = embedding_index.get(word)
      if word is not None:
        embedding_matrix[idx] = word
    return embedding_matrix

embedding_matrix = prepare_embedding_matrix(tokenizer.word_index, embedding_index, embedding_dim=300)

In [26]:
len(word_index)

119255

In [27]:
embedding_matrix[1]

array([-3.26099992e-01,  5.15600026e-01, -2.82099992e-01,  1.48000000e-02,
       -2.45299995e-01, -5.04000001e-02,  2.93000013e-01, -3.52299988e-01,
        3.30300003e-01, -9.64000002e-02,  4.39999998e-02,  2.78800011e-01,
       -2.74000000e-02, -1.46400005e-01,  2.95599997e-01,  5.33999987e-02,
        2.47600004e-01,  2.19400004e-01,  1.74300000e-01, -1.65099993e-01,
        2.15599999e-01, -1.58500001e-01,  2.78100014e-01, -8.99000019e-02,
       -1.29000004e-02, -7.86999986e-02, -7.11700022e-01,  1.56599998e-01,
       -1.14100002e-01,  2.22399995e-01,  1.78000003e-01, -3.10900003e-01,
        1.37899995e-01, -1.48000002e-01,  1.07299998e-01,  2.57900000e-01,
        7.41000026e-02, -3.45699996e-01,  1.49200007e-01,  6.26000017e-02,
        4.43300009e-01,  1.77100003e-01, -1.29800007e-01,  1.14200003e-01,
        2.41999999e-01, -8.20000023e-02, -2.97699988e-01,  3.59600008e-01,
       -1.18400000e-01, -2.30299994e-01, -2.40000011e-03,  4.17499989e-01,
        2.89200008e-01,  

Hàm kiểm tra lập trình

In [28]:
def test_embedding_matrix_shape():
    tokenizer_word_index = {'the': 1, 'cat': 2, 'sat': 3}
    embedding_index = {'the': np.random.rand(300), 'cat': np.random.rand(300)}
    embedding_dim = 300
    embedding_matrix = prepare_embedding_matrix(tokenizer_word_index, embedding_index, embedding_dim)
    assert embedding_matrix.shape == (len(tokenizer_word_index) + 1, embedding_dim), "Embedding matrix shape is incorrect."
    print('Test passed')

def test_embedding_vector_assignment():
    tokenizer_word_index = {'the': 1, 'cat': 2}
    the_vector = np.random.rand(300)
    cat_vector = np.random.rand(300)
    embedding_index = {'the': the_vector, 'cat': cat_vector}
    embedding_dim = 300
    embedding_matrix = prepare_embedding_matrix(tokenizer_word_index, embedding_index, embedding_dim)
    assert np.array_equal(embedding_matrix[1], the_vector), "'the' vector is not correctly assigned."
    assert np.array_equal(embedding_matrix[2], cat_vector), "'cat' vector is not correctly assigned."
    print('Test passed')

test_embedding_matrix_shape()
test_embedding_vector_assignment()

Test passed
Test passed


Kết quả mong đợi:

```
Test passed
Test passed
```

Kiểm tra chiều

In [29]:
embedding_matrix.shape

(119256, 300)

In [30]:
x_train.shape, x_test.shape

((40000, 200), (10000, 200))

Số lượng từ trong từ điển

In [31]:
vocab_size = len(word_index) + 1 #OOV

In [32]:
embedding_dim = 300

**TODO 6:** Xây dựng mô hình
- Mạng Bidirectional nhân LSTM sửa dụng lại Embedding và không train lại

In [33]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable = False, input_length = 200))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 300)          35776800  
                                                                 
 bidirectional (Bidirection  (None, 128)               186880    
 al)                                                             
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 35966801 (137.20 MB)
Trainable params: 190001 (742.19 KB)
Non-trainable params: 35776800 (136.48 MB)
_________________________________________________________________


In [35]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [36]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1250/1250 [==============================] - 41s 26ms/step - loss: 0.4525 - acc: 0.7775 - val_loss: 0.3182 - val_acc: 0.8605
Epoch 2/10
1250/1250 [==============================] - 30s 24ms/step - loss: 0.2886 - acc: 0.8817 - val_loss: 0.2882 - val_acc: 0.8822
Epoch 3/10
1250/1250 [==============================] - 27s 22ms/step - loss: 0.2630 - acc: 0.8938 - val_loss: 0.2718 - val_acc: 0.8888
Epoch 4/10
1250/1250 [==============================] - 25s 20ms/step - loss: 0.2382 - acc: 0.9032 - val_loss: 0.2592 - val_acc: 0.8926
Epoch 5/10
1250/1250 [==============================] - 33s 27ms/step - loss: 0.2144 - acc: 0.9147 - val_loss: 0.2557 - val_acc: 0.8963
Epoch 6/10
1250/1250 [==============================] - 37s 30ms/step - loss: 0.1876 - acc: 0.9268 - val_loss: 0.2751 - val_acc: 0.8926
Epoch 7/10
1250/1250 [==============================] - 27s 22ms/step - loss: 0.1626 - acc: 0.9378 - val_loss: 0.2802 - val_acc: 0.8897
Epoch 8/10
1250/1250 [==========================

**TODO 7:** Xây dựng mô hình
- Mạng Bidirectional nhân LSTM sửa dụng lại Embedding và train lại embedding

In [37]:
model_retrain_embedding = Sequential()
# Thêm lớp Embedding
model_retrain_embedding.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable = True, input_length = 200))
model_retrain_embedding.add(Bidirectional(LSTM(64)))
model_retrain_embedding.add(Dense(24, activation='relu'))
model_retrain_embedding.add(Dense(1, activation='sigmoid'))

model_retrain_embedding.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model_retrain_embedding.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
1250/1250 [==============================] - 165s 126ms/step - loss: 0.3958 - acc: 0.8285 - val_loss: 0.2887 - val_acc: 0.8885
Epoch 2/10
1250/1250 [==============================] - 70s 56ms/step - loss: 0.1903 - acc: 0.9297 - val_loss: 0.2526 - val_acc: 0.8992
Epoch 3/10
1250/1250 [==============================] - 50s 40ms/step - loss: 0.0811 - acc: 0.9736 - val_loss: 0.3371 - val_acc: 0.8956
Epoch 4/10
1250/1250 [==============================] - 52s 42ms/step - loss: 0.0308 - acc: 0.9911 - val_loss: 0.4031 - val_acc: 0.8914
Epoch 5/10
1250/1250 [==============================] - 49s 39ms/step - loss: 0.0146 - acc: 0.9956 - val_loss: 0.5859 - val_acc: 0.8804
Epoch 6/10
1250/1250 [==============================] - 43s 34ms/step - loss: 0.0078 - acc: 0.9973 - val_loss: 0.7533 - val_acc: 0.8823
Epoch 7/10
1250/1250 [==============================] - 43s 34ms/step - loss: 0.0063 - acc: 0.9980 - val_loss: 0.7663 - val_acc: 0.8792
Epoch 8/10
1250/1250 [========================

**TODO 8 - Kết luận:** Liệt kê kết quả valition tốt nhất đạt được

Model không Retrain embedding: 89.63%

Model có Retrain embedding: 89.92%



**Kết quả mong đợi (Tương đối):**

Model không Retrain embedding: 89.23 %

Model có Retrain embedding: 89.60 %